<a href="https://colab.research.google.com/github/IgorShestakov/GeoAnalytics/blob/Geo/GeoPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
GeoPoints = pd.read_csv('/content/drive/MyDrive/all_v2.csv')
GeoPoints.info()
GeoPoints.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5477006 entries, 0 to 5477005
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   price          int64  
 1   date           object 
 2   time           object 
 3   geo_lat        float64
 4   geo_lon        float64
 5   region         int64  
 6   building_type  int64  
 7   level          int64  
 8   levels         int64  
 9   rooms          int64  
 10  area           float64
 11  kitchen_area   float64
 12  object_type    int64  
dtypes: float64(4), int64(7), object(2)
memory usage: 543.2+ MB


,price,date,time,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type
0,6050000,2018-02-19,20:00:21,59.805808,30.376141,2661,1,8,10,3,82.6,10.8,1
1,8650000,2018-02-27,12:04:54,55.683807,37.297405,81,3,5,24,2,69.1,12.0,1
2,4000000,2018-02-28,15:44:00,56.295250,44.061637,2871,1,5,9,3,66.0,10.0,1
3,1850000,2018-03-01,11:24:52,44.996132,39.074783,2843,4,12,16,2,38.0,5.0,11
4,5450000,2018-03-01,17:42:43,55.918767,37.984642,81,3,13,14,2,60.0,10.0,1


Датасет имеет некорректные номера регионов.
Для получения корректных данных загрузим датасет с полигонами и названиями регионов.
Используя GeoPandas можно проверить входят ли координаты в полигон региона.


In [ ]:
Polygons = gpd.read_file('/content/drive/MyDrive/geo.json')
Polygons.info()
Polygons.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID_0       85 non-null     int64   
 1   ISO        85 non-null     object  
 2   NAME_0     85 non-null     object  
 3   ID_1       85 non-null     int64   
 4   NAME_1     85 non-null     object  
 5   NL_NAME_1  81 non-null     object  
 6   VARNAME_1  84 non-null     object  
 7   TYPE_1     85 non-null     object  
 8   ENGTYPE_1  85 non-null     object  
 9   geometry   85 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 6.8+ KB


,ID_0,ISO,NAME_0,ID_1,NAME_1,NL_NAME_1,VARNAME_1,TYPE_1,ENGTYPE_1,geometry
0,186,RUS,Russia,28,Karachay-Cherkess,?????????-????????? ??????????,Karaçay-Çerkes|Karachay-Cherkessiya|Karachayev...,Respublika,Republic,"POLYGON ((42.13194 44.34488, 42.19061 44.32248..."
1,186,RUS,Russia,29,Karelia,?????????? ???????,Karelian A.S.S.R.|Karelo-Finnish A.S.S.R.|Kare...,Respublika,Republic,"MULTIPOLYGON (((36.77098 64.02402, 36.75969 64..."
2,186,RUS,Russia,30,Kemerovo,??????????? ???????,Kemerovskaya Oblast,Oblast,Region,"POLYGON ((88.63731 56.77809, 88.64615 56.73922..."
3,186,RUS,Russia,31,Khabarovsk,??????????? ????,Khabarovskiy Kray,Kray,Territory,"MULTIPOLYGON (((136.64362 45.04694, 136.64417 ..."
4,186,RUS,Russia,32,Khakass,?????????? ???????,Khakassiya|Republic of Khakasia|Khakasskaya A....,Respublika,Republic,"MULTIPOLYGON (((91.45279 53.70172, 91.44386 53..."


In [ ]:
Polygons = Polygons.drop(['ID_0','ISO','NAME_0','ID_1','NL_NAME_1','VARNAME_1','ENGTYPE_1'], axis=1)

Создаем GeoDataFrame и формируем столбец с координатами, который будет иметь специальный статус. Этот GeoSeries называется «геометрией» GeoDataFrame. Когда пространственный метод применяется к GeoDataFrame, эти команды всегда действуют на столбец «geometry».


In [ ]:
GeoPoints = gpd.GeoDataFrame(geometry=gpd.points_from_xy(GeoPoints.geo_lon, GeoPoints.geo_lat, crs="EPSG:4326"), data=GeoPoints)
GeoPoints = GeoPoints.drop(['geo_lat', 'geo_lon','object_type', 'region'], axis=1)
GeoPoints.head()

,price,date,time,building_type,level,levels,rooms,area,kitchen_area,geometry
0,6050000,2018-02-19,20:00:21,1,8,10,3,82.6,10.8,POINT (30.37614 59.80581)
1,8650000,2018-02-27,12:04:54,3,5,24,2,69.1,12.0,POINT (37.29740 55.68381)
2,4000000,2018-02-28,15:44:00,1,5,9,3,66.0,10.0,POINT (44.06164 56.29525)
3,1850000,2018-03-01,11:24:52,4,12,16,2,38.0,5.0,POINT (39.07478 44.99613)
4,5450000,2018-03-01,17:42:43,3,13,14,2,60.0,10.0,POINT (37.98464 55.91877)


Используем метод sjoin (Spatial join), который позволяет проверить вхождение (within) точек в полигоны и подтянуть корректное поле регионов для каждой строки.

In [ ]:
GeoPoints = gpd.sjoin(GeoPoints, Polygons, predicate = 'within')
GeoPoints.head()

,price,date,time,building_type,level,levels,rooms,area,kitchen_area,geometry,index_right,NAME_1,TYPE_1
0,6050000,2018-02-19,20:00:21,1,8,10,3,82.6,10.8,POINT (30.37614 59.80581),23,City of St. Petersburg,Gorsovet
7,3600000,2018-03-04,20:52:38,1,2,5,1,31.1,6.0,POINT (30.39546 59.87553),23,City of St. Petersburg,Gorsovet
36,3200000,2018-04-10,16:10:12,1,7,9,1,31.0,7.0,POINT (30.20183 59.82746),23,City of St. Petersburg,Gorsovet
47,6500000,2018-04-23,11:47:58,3,3,3,3,89.0,10.0,POINT (29.78693 59.98833),23,City of St. Petersburg,Gorsovet
54,6300000,2018-04-28,17:47:24,3,5,5,2,99.9,14.5,POINT (30.28456 59.91162),23,City of St. Petersburg,Gorsovet


Обработанные данные можно загружать в любую BI-систему, поддерживающую работу с картами. Ниже я попробую обработать датасеты через библиотеки Python.